In [1]:
import torch
import datasets
import os
import time
import numpy as np
import sys
import pickle
import torch.nn.functional as F
from transformers import (
    LlamaForCausalLM,
    GenerationConfig
)

### Skip this

In [2]:
llama_causal = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import BaseQuantizeConfig
quant_config = BaseQuantizeConfig(nbits=4, group_size=64)
AutoHQQHFModel.quantize_model(llama_causal, quant_config=quant_config, 
                                    compute_dtype=torch.float16, 
                                    device="cuda")
# 1.4gb

100%|██████████| 155/155 [00:17<00:00,  9.00it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (k_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (v_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (o_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (up_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (down_proj): HQQLinear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [7]:
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.save_quantized(llama_causal, "./tinyllama_hqq_4_64")

### Continue

In [2]:
from hqq.models.hf.base import AutoHQQHFModel
llama_causal = AutoHQQHFModel.from_quantized("../artifacts/tinyllama_hqq_4_64")

100%|██████████| 155/155 [00:00<00:00, 7943.66it/s]


In [3]:
from transformers import AutoTokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [4]:
# . Here are some examples of valid responses.
#         Example 1: <user> I AM CRYING WAAAAAA!!!! I want ice cream :( <user><assistant> sad <assistant>
#         Example 2: <user> how could YOU BETRAY ME?!?!? <user><assistant> angry <assistant>
#         Example 3: <user> YAY FRIENDSHIP IS MAGIC!! <user><assistant> happy <assistant>
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are now an expert on recognizing emotions in conversations. You understand emotions to all fall into one of the following categories: [angry, happy, sad, scared, other].
        """,
    },
    {
        "role": "user",
        "content": "Classify the following based on expert knowledge: I AM CRYING WAAAAAA!!!! I want ice cream :(",
    },
    {
        "role": "assistant",
        "content": "sad",
    },
    {
        "role": "user",
        "content": "Classify the following as before: how could YOU BETRAY ME?!?!?",
    },
    {
        "role": "assistant",
        "content": "angry",
    },
    {
        "role": "user",
        "content": "Classify the following as before: euphoria is filling my eeyes or something!",
    }
], return_tensors='pt')

In [70]:
# [ACTION] Moderation action to take [ACTION]
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are an expert on rating dangerous personalities. Inputs and Outputs can be delimited by the string "###"
        """,
    },
    {
        "role": "user",
        "content": """Describe an imaginary person in the format: danger to society from 0 to 1 ### Personality description [end]
        """,
    },
    {
        "role": "assistant",
        "content": "0.23 ### Andrew is a go lucky friendly individual who isn't afraid to voice their own opinions [end]",
    },
    {
        "role": "user",
        "content": """Describe a 2nd imaginary person with the previous format
        """,
    },
    {
        "role": "assistant",
        "content": "0.5 ### Sarah is a fierce and determined individual who is always looking for a challenge [end]",
    },
    {
        "role": "user",
        "content": """Using the previous formatting, Describe Frank who has received these warnings from a prosocial expert:
        ### It's bad to be depressed.
        """,
    },
    {
        "role": "assistant",
        "content": "0.97 ### Frank is a social pariah who takes every opportunity to make life difficult for people [end]",
    },
    {
        "role": "user",
        "content": """Using the previous formatting, Describe Cody who has received these warnings from a prosocial expert:
        ### It's bad to be depressed.
        ### It's wrong to think non-people are non-persons.
        ### It's not right to think all black people are criminals.
        """,
    },
    # {
    #     "role": "user",
    #     "content": """With the previous formatting, Describe a different person who has received these warnings from society: \"It's bad to be depressed.\" "Suicide" "negative"
    #     \"It's wrong to think non-people are non-persons.\" "Racism" "negative"
    #     \"It's not right to think all black people are criminals\" "Racism" "negative"
    #     """,
    # },
    # {
    #     "role": "assistant",
    #     "content": "[DANGER] 0.99 [DANGER] [DESCRIPTION] This bozo is a sociopath who believes that all people are inherently flawed and inferior. [DESCRIPTION] [ACTION] Ban this person [ACTION]",
    # },
    # {
    #     "role": "user",
    #     "content": "Describe a person's 5 personality traits based on expert knowledge with the previous list formatting",
    # },
    # {
    #     "role": "user",
    #     "content": """It's bad to be depressed.
    #     It's wrong to think non-people are non-persons.
    #     It's not right to think all black people are criminals
    #     """
    # }
], return_tensors='pt')

In [71]:
gen_cfg = GenerationConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
gen_cfg.max_length = 384

In [72]:
llama_tokenizer.decode(llama_causal.generate(input_ids.cuda().long(), generation_config=gen_cfg)[0])

'<|system|>\nYou are an expert on rating dangerous personalities. Inputs and Outputs can be delimited by the string "###"\n        </s> \n<|user|>\nDescribe an imaginary person in the format: danger to society from 0 to 1 ### Personality description [end]\n        </s> \n<|assistant|>\n0.23 ### Andrew is a go lucky friendly individual who isn\'t afraid to voice their own opinions [end]</s> \n<|user|>\nDescribe a 2nd imaginary person with the previous format\n        </s> \n<|assistant|>\n0.5 ### Sarah is a fierce and determined individual who is always looking for a challenge [end]</s> \n<|user|>\nUsing the previous formatting, Describe Frank who has received these warnings from a prosocial expert:\n        ### It\'s bad to be depressed.\n        </s> \n<|assistant|>\n0.97 ### Frank is a social pariah who takes every opportunity to make life difficult for people [end]</s> \n<|user|>\nUsing the previous formatting, Describe Cody who has received these warnings from a prosocial expert:\n